In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 라이브러리 가져오기
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

In [3]:
# GPU 사용설정
device='cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
# parameter 설정
batch_size=100
num_epochs=5
learning_rate=0.001

In [5]:
# 데이터셋 설정
transform = transforms.Compose([
    transforms.Resize((90, 120)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

train_data = torchvision.datasets.ImageFolder(
    root='drive/MyDrive/Colab/custom_data_chair/train_data/',
    transform=transform
)

In [6]:
# dataloader 만들기
data_loader = DataLoader(
    dataset=train_data,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

In [7]:
# 학습모델 만들기
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer3 = nn.Sequential(
            nn.Linear(8208, 120),
            nn.ReLU(),
            nn.Linear(120, 2)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out

In [8]:
# 모델 선언
model=CNN().to(device)

In [9]:
# Loss function & Optimizer
criterion=nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [10]:
# Training
total_batch=len(data_loader)

for epoch in range(num_epochs):
    avg_cost=0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        hypothesis=model(imgs)
        cost=criterion(hypothesis,labels)
        cost.backward()
        optimizer.step()
        avg_cost+=cost/total_batch
    print('Epoch: {} Cost: {}'.format(epoch,cost))
print('Finished')

Epoch: 0 Cost: 0.6520465612411499
Epoch: 1 Cost: 0.47999244928359985
Epoch: 2 Cost: 0.284376323223114
Epoch: 3 Cost: 0.13391172885894775
Epoch: 4 Cost: 0.10642539709806442
Finished


In [12]:
# training 끝난 모델 state 저장
torch.save(model.state_dict(),'drive/MyDrive/Colab/cnn_chair_model_grayScale.pth')

In [13]:
# Test용 dataset 설정
transform = transforms.Compose([
    transforms.Resize((90, 120)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

test_data = torchvision.datasets.ImageFolder(
    root='drive/MyDrive/Colab/custom_data_chair/test_data/',
    transform=transform
)

In [14]:
# Test용 dataloader 만들기
test_data_loader=DataLoader(
    dataset=test_data,
    batch_size=len(test_data)
)

In [15]:
# 테스트 끝난 모델 가져오기
new_net=CNN().to(device)
new_net.load_state_dict(torch.load('drive/MyDrive/Colab/cnn_chair_model_grayScale.pth'))

<All keys matched successfully>

In [17]:
# Test
with torch.no_grad():
    for num, data in enumerate(test_data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)

        prediction = new_net(imgs)
        correct_prediction = torch.argmax(prediction, 1) == labels
        accuracy = correct_prediction.float().mean()
        print("Accuracy: {}".format(accuracy.item()))

Accuracy: 0.9599999785423279
